In [1]:
import os
import re
import glob
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm_notebook

try:
    from jupyterthemes import jtplot
    jtplot.style(theme='onedork', context='talk', fscale=1.4, ticks=True, grid=True, figsize=(10, 6))
except ModuleNotFoundError:
    print("Jupyterthemes not found. Check https://github.com/dunovank/jupyter-themes for more information.")

In [2]:
ROOT = '../.data/news-kor-large/'
assert os.path.isdir(ROOT), "Check the root directory."

In [4]:
filenames = glob.glob(os.path.join(ROOT, '**/*/data.csv'))
print(f"{len(filenames)} .csv files available.")

23 .csv files available.


In [5]:
filenames

['../.data/news-kor-large/경제/글로벌경제/data.csv',
 '../.data/news-kor-large/경제/금융/data.csv',
 '../.data/news-kor-large/경제/부동산/data.csv',
 '../.data/news-kor-large/경제/산업_재계/data.csv',
 '../.data/news-kor-large/경제/생활경제/data.csv',
 '../.data/news-kor-large/경제/중기_벤처/data.csv',
 '../.data/news-kor-large/경제/증권/data.csv',
 '../.data/news-kor-large/사회/교육/data.csv',
 '../.data/news-kor-large/사회/노동/data.csv',
 '../.data/news-kor-large/사회/사건사고/data.csv',
 '../.data/news-kor-large/사회/사회 일반/data.csv',
 '../.data/news-kor-large/사회/식품_의료/data.csv',
 '../.data/news-kor-large/사회/언론/data.csv',
 '../.data/news-kor-large/사회/인권_복지/data.csv',
 '../.data/news-kor-large/사회/인물/data.csv',
 '../.data/news-kor-large/사회/지역/data.csv',
 '../.data/news-kor-large/사회/환경/data.csv',
 '../.data/news-kor-large/정치/국방_외교/data.csv',
 '../.data/news-kor-large/정치/국회_정당/data.csv',
 '../.data/news-kor-large/정치/북한/data.csv',
 '../.data/news-kor-large/정치/정치일반/data.csv',
 '../.data/news-kor-large/정치/청와대/data.csv',
 '../.data/news-kor-la

In [6]:
for filename in filenames: 
    with open(filename, 'r') as f:
        lines = f.readlines()
    break

In [7]:
len(lines)

855775

In [13]:
print(*lines[:500], sep='\n')

category1,category2,contents,date,news,time,title

경제,글로벌경제,"









// flash 오류를 우회하기 위한 함수 추가

function _flash_removeCallback() {}



 2015년 마지막 거래일인 지난달 31일(현지시간) 미국 주식시장이 하락하며 글로벌 금융위기 이후 최악의 성적을 냈다. 2008년 글로벌 금융위기 이후 처음으로 연간 기준 수익률이 마이너스(-) 손실을 기록한 것이다. 지난달 31일 S&P500은 전일 대비 0.9% 떨어지면서 연간 기준으로 0.7% 하락 마감했다. 이는 2008년 이후 첫 손실이다. 2008년 글로벌 금융위기 당시 S&P500은 전년 대비 38.5% 급락했다. 하지만 2009년부터는 6년 연속 상승했다.대형주 위주인 미국 다우존스지수도 지난달 31일 1.02% 하락해 1만7425.03에 거래를 마쳤다. 연간 기준으로 보면 2.2% 낮아져 2008년 이후 첫 마이너스 수익률을 냈다. 반면 기술주 중심의 나스닥지수는 58.44포인트(1.15%) 낮은 5007.41에 장을 마쳤지만 지난 한 해 5.7%나 올랐다.미국 주식시장은 지난해 상반기만 해도 사상 최고치를 기록하면서 미국 경제의 회복세를 반영했다. 하지만 중국 경기 둔화와 미국 연준 기준금리 인상에 따른 우려가 커지면서 작년 8월부터 급락세를 보였다. 전문가들은 지난해 12월 미국 기준금리 인상과 유가 폭락으로 인한 에너지 기업 주가 하락을 원인으로 꼽고 있다. 6년간 지속됐던 미국 주식시장 랠리가 마침내 끝에 도달했다는 분석도 나온다. 미국 주식시장 마지막 날 하락세는 대형주들이 주도했다. 애플은 1.92% 하락해 다우지수 구성 종목 중 가장 크게 떨어졌다. 애플도 연간 기준으로 2008년 이후 처음 하락했다.지난해 4월 장중 주당 134.54달러까지 오르며 사상 최고치를 기록했던 애플은 이후 하락세로 돌아섰다. 신제품인 아이폰6s가 성공을 거뒀지만 스마트폰 시장 정체로 더 이상 성장 모멘텀을 찾

In [10]:
lines[0]

'category1,category2,contents,date,news,time,title\n'

In [16]:
lines[9]

' 2015년 마지막 거래일인 지난달 31일(현지시간) 미국 주식시장이 하락하며 글로벌 금융위기 이후 최악의 성적을 냈다. 2008년 글로벌 금융위기 이후 처음으로 연간 기준 수익률이 마이너스(-) 손실을 기록한 것이다. 지난달 31일 S&P500은 전일 대비 0.9% 떨어지면서 연간 기준으로 0.7% 하락 마감했다. 이는 2008년 이후 첫 손실이다. 2008년 글로벌 금융위기 당시 S&P500은 전년 대비 38.5% 급락했다. 하지만 2009년부터는 6년 연속 상승했다.대형주 위주인 미국 다우존스지수도 지난달 31일 1.02% 하락해 1만7425.03에 거래를 마쳤다. 연간 기준으로 보면 2.2% 낮아져 2008년 이후 첫 마이너스 수익률을 냈다. 반면 기술주 중심의 나스닥지수는 58.44포인트(1.15%) 낮은 5007.41에 장을 마쳤지만 지난 한 해 5.7%나 올랐다.미국 주식시장은 지난해 상반기만 해도 사상 최고치를 기록하면서 미국 경제의 회복세를 반영했다. 하지만 중국 경기 둔화와 미국 연준 기준금리 인상에 따른 우려가 커지면서 작년 8월부터 급락세를 보였다. 전문가들은 지난해 12월 미국 기준금리 인상과 유가 폭락으로 인한 에너지 기업 주가 하락을 원인으로 꼽고 있다. 6년간 지속됐던 미국 주식시장 랠리가 마침내 끝에 도달했다는 분석도 나온다. 미국 주식시장 마지막 날 하락세는 대형주들이 주도했다. 애플은 1.92% 하락해 다우지수 구성 종목 중 가장 크게 떨어졌다. 애플도 연간 기준으로 2008년 이후 처음 하락했다.지난해 4월 장중 주당 134.54달러까지 오르며 사상 최고치를 기록했던 애플은 이후 하락세로 돌아섰다. 신제품인 아이폰6s가 성공을 거뒀지만 스마트폰 시장 정체로 더 이상 성장 모멘텀을 찾기 어렵다는 분석 때문이다.[이덕주 기자][ⓒ 매일경제 & mk.co.kr, 무단전재 및 재배포 금지]\n'

In [12]:
lines[2]

'\n'